# <center> Structured Query Language (SQL) </center>

### Lenguaje de definición de datos (DDL)
* CREATE
* ALTER
* DROP
* TRUNCATE

### Lenguaje de manipulación de datos (DML)
* SELECT
* INSERT
* UPDATE
* DELETE

### DB Estrellas
(tipo Leo Maslíah)
<br>
<center>
<img src="db02.png" width="100%">
</center>



### CREATE TABLE
```SQL
CREATE TABLE estrellas (
    nombre      VARCHAR        PRIMARY KEY,
    tipo        VARCHAR,
    descubierta DATE,
    puntas      NUMERIC (2, 2),
    ubicacion   VARCHAR
);
```

### SELECT - forma básica

```SQL
SELECT [ALL | DISTINCT ]

             <nombre_campo> [{,<nombre_campo>}]

FROM <nombre_tabla>|<nombre_vista> 

        [{,<nombre_tabla>|<nombre_vista>}]

[WHERE <condición> [{ AND|OR <condición>}]]

[GROUP BY <nombre_campo> [{,<nombre_campo >}]]

[HAVING <condición>[{ AND|OR <condición>}]]

[ORDER BY <nombre_campo>|<indice_campo> [ASC | DESC]

                [{,<nombre_campo>|<indice_campo> [ASC | DESC ]}]]
```

In [ ]:
import sqlite3
conn = sqlite3.connect('/home/ag/DBs/estrellas')
conn.row_factory = sqlite3.Row
dbcur = conn.cursor()

In [ ]:
q = """SELECT *
       FROM estrellas
       """
print("nombre - tipo - descubierta - puntas - ubicación")
print("------------------------------------------------")
for row in dbcur.execute(q):
    print(row[0], " - ", row[1], " - ", row[2], " - ", row[3], " - ", row[4])

### Con Pandas

In [ ]:
import pandas as pd
from IPython.display import display, HTML
q = """SELECT *
       FROM estrellas
       """
df = pd.read_sql_query(q, conn)
display(HTML(df.to_html(index=False)))

### Alias y selección de atributos

In [ ]:
q = """SELECT nombre as Nombre, E.puntas as Puntas
       FROM estrellas as E
       """
df = pd.read_sql_query(q, conn)
display(HTML(df.to_html(index=False)))

### Cláusula WHERE

In [ ]:
q = """SELECT *
       FROM estrellas
       WHERE puntas = '5' OR nombre LIKE 'De%'
       """
df = pd.read_sql_query(q, conn)
display(HTML(df.to_html(index=False)))

### Cláusula ORDER BY

In [ ]:
q = """SELECT *
       FROM estrellas
       ORDER BY puntas ASC, nombre DESC
       """
df = pd.read_sql_query(q, conn)
display(HTML(df.to_html(index=False)))

### Cláusula GROUP BY

In [ ]:
q = """SELECT tipo/*, nombre*/
       FROM estrellas
       GROUP BY tipo
       """
df = pd.read_sql_query(q, conn)
display(HTML(df.to_html(index=False)))

### SELECT DISTINCT

In [ ]:
q = """SELECT DISTINCT tipo
       FROM estrellas
       """
df = pd.read_sql_query(q, conn)
display(HTML(df.to_html(index=False)))

### Cláusula GROUP BY (SUM, COUNT, AVG, ...)

In [ ]:
q = """SELECT tipo, COUNT(puntas) as cantPtas, COUNT(DISTINCT puntas) as cantDistPtas, 
                    AVG(puntas) as avgPtas, SUM(puntas) as sumPtas, MAX(nombre) as maxNom, 
                    count(*) as cantTotal
       FROM estrellas
       GROUP BY tipo
       """
df = pd.read_sql_query(q, conn)
display(HTML(df.to_html(index=False)))

### Cláusula HAVING

In [ ]:
q = """SELECT tipo, COUNT(puntas) as cantPtas, COUNT(DISTINCT puntas) as cantDistPtas, AVG(puntas) as avgPtas, 
              SUM(puntas) as sumPtas, MAX(nombre) as maxNom, count(*) as cantTotal
       FROM estrellas
       GROUP BY tipo
       HAVING cantPtas = 2
       """
df = pd.read_sql_query(q, conn)
display(HTML(df.to_html(index=False)))

### Cláusula HAVING

In [ ]:
q = """SELECT tipo, COUNT(puntas) as cantPtas, COUNT(DISTINCT puntas) as cantDistPtas, AVG(puntas) as avgPtas, 
              SUM(puntas) as sumPtas, MAX(nombre) as maxNom, count(*) as cantTotal
       FROM estrellas
       GROUP BY tipo
       HAVING MIN(ubicacion) = 'Bs. As.'
       """
df = pd.read_sql_query(q, conn)
display(HTML(df.to_html(index=False)))

### Consultas multitabla

In [ ]:
q = """SELECT E.nombre Nombre, T.descripcion as Tipo
       FROM estrellas E, tipos T
       /*WHERE E.tipo = T.tipo*/
       """
df = pd.read_sql_query(q, conn)
display(HTML(df.to_html(index=False)))

### Consultas multitabla

Cantidad de producciones y valoración media por tipo de Estrella

In [ ]:
q = """SELECT T.descripcion as Tipo, AVG(I.val) as Val_avg, COUNT(*) as Prod_cant
       FROM estrellas as E, tipos as T, imdb as I, imdb_est as IE
       WHERE E.tipo = T.tipo
       AND   E.nombre = IE.nombre
       AND   IE.id_prod = I.id_prod
       GROUP BY T.descripcion """
df = pd.read_sql_query(q, conn)
display(HTML(df.to_html(index=False)))

### Consultas multitabla
Tablas repetidas en el FROM

In [ ]:
q = """SELECT E1.nombre as Nombre_A, E2.nombre as Nombre_B, E1.descubierta as Fecha
       FROM estrellas as E1, estrellas E2
       WHERE E1.descubierta = E2.descubierta
       AND   E1.nombre < E2.nombre
       """
df = pd.read_sql_query(q, conn)
display(HTML(df.to_html(index=False)))

### Subconsultas

In [ ]:
q = """SELECT IE.nombre, I.prod
       FROM imdb as I, imdb_est as IE
       WHERE  IE.id_prod = I.id_prod
       AND    IE.nombre IN (SELECT nombre FROM estrellas WHERE puntas > 5) 
       """
df = pd.read_sql_query(q, conn)
display(HTML(df.to_html(index=False)))

### Subconsultas

In [ ]:
q = """SELECT S2.descripcion as Tipo, AVG(I.val) as Val_avg, COUNT(*) as Prod_cant
       FROM (SELECT S1.descripcion, IE.Id_prod
             FROM (SELECT T.descripcion, E.nombre
                   FROM estrellas as E, tipos as T
                   WHERE E.tipo = T.tipo) as S1,
                   imdb_est as IE
             WHERE S1.nombre = IE.nombre) aS S2,
             imdb as I
       WHERE S2.id_prod = I.id_prod
       GROUP BY S2.descripcion """
df = pd.read_sql_query(q, conn)
display(HTML(df.to_html(index=False)))

### INSERT - Forma básica

```SQL
INSERT INTO <nombre_tabla> (<nombre_campo_A>, [<nombre_campo_B>, ... ])
VALUES (<valor_A>, [<valor_B>, ...])
```
O también:
```SQL
INSERT INTO <nombre_tabla> 
VALUES (<valor_A>, <valor_B>, ...)
```
O también:
```SQL
INSERT INTO <nombre_tabla> 
SELECT...
```

### INSERT - Ejemplo

In [ ]:
q = """INSERT INTO muestras (archivo, origen, Fs, Fs_orig, tipo, EGG, habla, vocal, F0, 
                             np)
       VALUES (?, 'lr01', '25 kHz', ?, 'Mono', 'No', 'vocal sostenida', ?, -1, 'noHUPA')
       """
params = [i, fs_orig, vocal]
dbcur.execute(q, params)
conn.commit()

### UPDATE - Forma básica

```SQL
UPDATE <nombre_tabla> 
SET <nombre_campo_A> = <valor_A>,
    <nombre_campo_B> = <valor_B>, ...
WHERE <nombre_campo_C> = <valor_C>
```

### DELETE - Forma básica

```SQL
DELETE 
FROM <nombre_tabla> 
WHERE <nombre_campo_C> = <valor_C>
```

### <center>THE END<center/>
<br>
<center>
<img src="chuck_norris.jpg" width="30%">
</center>    